In [5]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.2.0-bin-hadoop3.2'

In [6]:
import pyspark

In [7]:
from pyspark import SparkContext
sc=SparkContext("local","Test-app")

In [8]:
import math
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Distancia de Minkowski

In [93]:
numeros=[(2,1),(3,1),(8,2)]
p=2
datos=sc.parallelize(numeros).zipWithIndex()

mape=datos.flatMap(lambda x:[(x[1],i) for i in x[0]])
reduce1=mape.reduceByKey(lambda x,y:(math.fabs(x-y))**p)
mape1=reduce1.flatMap(lambda x:[(x[1]) for i in x]).distinct()
reduce2=mape1.reduce(lambda x,y:(x+y))
rdd=reduce2**(1/p)
print(rdd)


6.4031242374328485


# Escalonamiento

In [54]:
numeros2=[(9,6,7)]
datos=sc.parallelize(numeros2)
maximo=max(datos.first())
minimo=min(datos.first())
mape=datos.map(lambda x:[(i-minimo)/(maximo-minimo) for i in x])
print(mape.collect())

[[1.0, 0.0, 0.3333333333333333]]


# Normalizacion

In [95]:
numeros2=[(1,1,1)]
datos=sc.parallelize(numeros2)
modulo=datos.flatMap(lambda x:((i**2) for i in x))
reduce=modulo.reduce(lambda x,y:(x+y))
mape=datos.flatMap(lambda x:[(i/(reduce)**0.5) for i in x])
print(mape.collect())

[0.5773502691896258, 0.5773502691896258, 0.5773502691896258]


# TF

In [110]:
import math
from pyspark.sql.functions import *
#Creamos un corpus de 2 dimensiones
corpus=["Profe disculpe la tardanza ","la UNSAAC es  la mejor universidad"]
#al corpus le damos un ID autoincrementable con la funcion zipWithIndex
data=sc.parallelize(corpus).zipWithIndex()
#Calculamos la cantidad de elementos dentro de cada oracion, para poder calcualar el TF
#Ejemplo en "messi messi messi ronaldo ronaldo balon" hay 6 palabras
map0=data.flatMap(lambda x: [((x[1],x[0]),1) for i in x[0].split()])
reduce0=map0.reduceByKey(lambda x,y:x+y)
#Guardamos  el ID de la oracion, el total de elementos de la oracion y la cantidad de apariciones de una palabra
map1=reduce0.flatMap(lambda x: [((x[0][0],x[1],i),1) for i in x[0][1].split()])
reduce1=map1.reduceByKey(lambda x,y:x+y)
#Calculamos el TF
tf=reduce1.map(lambda x: (x[0][2],(1+math.log(x[1]/x[0][1],10),x[0][0])))
print("EL ID ES EL IDENTIFICADOR, QUE NOS INDICA EN QUE NUMERO DE ORACION HA APARECIDO ESTA PALABRA")
print("ID  -  WORD    -        TF   ")
for x in tf.collect():
    print(f'{x[1][1]} :  {x[0]} :      {x[1][0] }' ) 


EL ID ES EL IDENTIFICADOR, QUE NOS INDICA EN QUE NUMERO DE ORACION HA APARECIDO ESTA PALABRA
ID  -  WORD    -        TF   
0 :  Profe :      0.3979400086720377
0 :  disculpe :      0.3979400086720377
0 :  la :      0.3979400086720377
0 :  tardanza :      0.3979400086720377
1 :  la :      0.5228787452803376
1 :  UNSAAC :      0.22184874961635648
1 :  es :      0.22184874961635648
1 :  mejor :      0.22184874961635648
1 :  universidad :      0.22184874961635648


# IDF

In [109]:
import math
from pyspark.sql.functions import *
#Creamos un corpus de 2 dimensiones
corpus=["Profe disculpe la tardanza ","la UNSAAC es  la mejor universidad"]
#al corpus le damos un ID autoincrementable con la funcion zipWithIndex
data=sc.parallelize(corpus).zipWithIndex()
#Calculamos la cantidad de elementos dentro de cada oracion, para poder calcualar el TF
#Ejemplo en "messi messi messi ronaldo ronaldo balon" hay 6 palabras
map0=data.flatMap(lambda x: [((x[1],x[0]),1) for i in x[0].split()])
reduce0=map0.reduceByKey(lambda x,y:x+y)
#Guardamos  el ID de la oracion, el total de elementos de la oracion y la cantidad de apariciones de una palabra
map1=reduce0.flatMap(lambda x: [((x[0][0],x[1],i),1) for i in x[0][1].split()])
reduce1=map1.reduceByKey(lambda x,y:x+y)
#Guardamos la palabra si es que aparece como minimo en una oracion acompañado de un 1
#para luego poder determinar en cuantas oraciones aparece esta palabra
map2=reduce1.flatMap(lambda x: [(x[0][2],1)] )
reduce2=map2.reduceByKey(lambda x,y:x+y)

#Calculamos el total de oraciones del corpus
total=data.count()

#Calculamos el idf
idf=reduce2.map(lambda x: (x[0],math.log(1+total/x[1],10)))

#MOSTRAMOS LOS RESULTADOS
print("EL ID ES EL IDENTIFICADOR, QUE NOS INDICA EN QUE NUMERO DE ORACION HA APARECIDO ESTA PALABRA")
print("ID  -  WORD    -       IDF    ")
for x in idf.collect():
    print(f' {x[0]} :        {x[1] }' ) 

EL ID ES EL IDENTIFICADOR, QUE NOS INDICA EN QUE NUMERO DE ORACION HA APARECIDO ESTA PALABRA
ID  -  WORD    -       IDF    
 Profe :        0.47712125471966244
 disculpe :        0.47712125471966244
 la :        0.30102999566398114
 tardanza :        0.47712125471966244
 UNSAAC :        0.47712125471966244
 es :        0.47712125471966244
 mejor :        0.47712125471966244
 universidad :        0.47712125471966244


# TF-IDF

In [ ]:
import math
from pyspark.sql.functions import *
#Creamos un corpus de 2 dimensiones
corpus=["Profe disculpe la tardanza ","la UNSAAC es  la mejor universidad"]
#al corpus le damos un ID autoincrementable con la funcion zipWithIndex
data=sc.parallelize(corpus).zipWithIndex()
#Calculamos la cantidad de elementos dentro de cada oracion, para poder calcualar el TF
#Ejemplo en "messi messi messi ronaldo ronaldo balon" hay 6 palabras
map0=data.flatMap(lambda x: [((x[1],x[0]),1) for i in x[0].split()])
reduce0=map0.reduceByKey(lambda x,y:x+y)
#Guardamos  el ID de la oracion, el total de elementos de la oracion y la cantidad de apariciones de una palabra
map1=reduce0.flatMap(lambda x: [((x[0][0],x[1],i),1) for i in x[0][1].split()])
reduce1=map1.reduceByKey(lambda x,y:x+y)
#Calculamos el TF
tf=reduce1.map(lambda x: (x[0][2],(1+math.log(x[1]/x[0][1],10),x[0][0])))

#Guardamos la palabra si es que aparece como minimo en una oracion acompañado de un 1
#para luego poder determinar en cuantas oraciones aparece esta palabra
map2=reduce1.flatMap(lambda x: [(x[0][2],1)] )
reduce2=map2.reduceByKey(lambda x,y:x+y)

#Calculamos el total de oraciones del corpus
total=data.count()

#Calculamos el idf
idf=reduce2.map(lambda x: (x[0],math.log(1+total/x[1],10)))

#Hacemos un join
rdd=tf.join(idf)

#Por ultimos guardamos dentro de "rdd" los elementos obtenidos
# con el siguiente esquema
#            Identificador       ,        Word ,    Tf   ,   IDF ,   TF*IDF 
rdd=rdd.map(lambda x: (x[1][0][1],x[0],(x[1][0][0],x[1][1],x[1][0][0]*x[1][1])))

#MOSTRAMOS LOS RESULTADOS
print("EL ID ES EL IDENTIFICADOR, QUE NOS INDICA EN QUE NUMERO DE ORACION HA APARECIDO ESTA PALABRA")
print("ID  -  WORD    -      TF        -         IDF        -        TF-IDF")
for x in rdd.collect():
    print(f'{x[0]+1} :  {x[1]} : {x[2] }' ) 

EL ID ES EL IDENTIFICADOR, QUE NOS INDICA EN QUE NUMERO DE ORACION HA APARECIDO ESTA PALABRA
ID  -  WORD    -      TF        -         IDF        -        TF-IDF
1 :  disculpe : (0.3979400086720377, 0.47712125471966244, 0.18986563624075598)
1 :  tardanza : (0.3979400086720377, 0.47712125471966244, 0.18986563624075598)
2 :  mejor : (0.22184874961635648, 0.47712125471966244, 0.10584875377494424)
2 :  universidad : (0.22184874961635648, 0.47712125471966244, 0.10584875377494424)
1 :  Profe : (0.3979400086720377, 0.47712125471966244, 0.18986563624075598)
1 :  la : (0.3979400086720377, 0.30102999566398114, 0.11979187908506814)
2 :  la : (0.5228787452803376, 0.30102999566398114, 0.15740218642452794)
2 :  UNSAAC : (0.22184874961635648, 0.47712125471966244, 0.10584875377494424)
2 :  es : (0.22184874961635648, 0.47712125471966244, 0.10584875377494424)
